In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer once
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.chains import APIChain
from langchain.llms import HuggingFaceLLM

# Initialize the model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Define API functions
def get_weather(location: str) -> dict:
    api_key = "YOUR_API_KEY"
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    complete_url = f"{base_url}?q={location}&appid={api_key}"
    response = requests.get(complete_url)
    return response.json()

def get_news(topic: str) -> dict:
    api_key = "YOUR_API_KEY"
    base_url = "https://newsapi.org/v2/top-headlines"
    complete_url = f"{base_url}?q={topic}&apiKey={api_key}"
    response = requests.get(complete_url)
    return response.json()

# Define the API chains
weather_chain = APIChain.from_llm_and_api_docs(llm, get_weather)
news_chain = APIChain.from_llm_and_api_docs(llm, get_news)

# Extract intent and entities
def extract_intent_and_entities(prompt: str) -> (str, dict):
    llm_prompt = (
        f"Extract the intent and entities from the following text: '{prompt}'.\n\n"
        "Intent: [INTENT]\n"
        "Entities: [ENTITIES]"
    )
    inputs = tokenizer(llm_prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    intent = None
    entities = {}
    for line in response.split('\n'):
        if line.startswith("Intent:"):
            intent = line.split(":")[1].strip()
        if line.startswith("Entities:"):
            entities_str = line.split(":")[1].strip()
            for entity in entities_str.split(','):
                key, value = entity.split('=')
                entities[key.strip()] = value.strip()
    
    return intent, entities

# Process user input
def process_user_input(prompt: str) -> str:
    intent, entities = extract_intent_and_entities(prompt)
    
    if intent == "weather_query":
        location = entities.get("location")
        if not location:
            return "Sorry, I couldn't identify the location. Please provide a valid location."
        weather_data = get_weather(location)
        if weather_data.get("weather"):
            description = weather_data["weather"][0]["description"]
            temp = weather_data["main"]["temp"]
            return f"The weather in {location} is currently {description} with a temperature of {temp}°K."
        else:
            return "Sorry, I couldn't fetch the weather data. Please check the location and try again."
    
    elif intent == "news_query":
        topic = entities.get("topic")
        if not topic:
            return "Sorry, I couldn't identify the topic. Please provide a valid topic."
        news_data = get_news(topic)
        if news_data.get("articles"):
            headlines = [article["title"] for article in news_data["articles"][:5]]
            return f"Top news headlines for {topic}:\n" + "\n".join(headlines)
        else:
            return "Sorry, I couldn't fetch the news data. Please check the topic and try again."
    
    else:
        return "Sorry, I can only help with weather and news queries."

# Example usage
user_prompt = "What's the weather in New York?"
response = process_user_input(user_prompt)
print(response)

user_prompt = "Tell me the latest news about technology."
response = process_user_input(user_prompt)
print(response)

In [ ]:
import asyncio
import aiohttp

async def get_weather(location: str) -> dict:
    api_key = "YOUR_API_KEY"
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    complete_url = f"{base_url}?q={location}&appid={api_key}"
    async with aiohttp.ClientSession() as session:
        async with session.get(complete_url) as response:
            return await response.json()

async def get_news(topic: str) -> dict:
    api_key = "YOUR_API_KEY"
    base_url = "https://newsapi.org/v2/top-headlines"
    complete_url = f"{base_url}?q={topic}&apiKey={api_key}"
    async with aiohttp.ClientSession() as session:
        async with session.get(complete_url) as response:
            return await response.json()

async def process_user_input(prompt: str) -> str:
    intent, entities = extract_intent_and_entities(prompt)
    
    if intent == "weather_query":
        location = entities.get("location")
        if not location:
            return "Sorry, I couldn't identify the location. Please provide a valid location."
        weather_data = await get_weather(location)
        if weather_data.get("weather"):
            description = weather_data["weather"][0]["description"]
            temp = weather_data["main"]["temp"]
            return f"The weather in {location} is currently {description} with a temperature of {temp}°K."
        else:
            return "Sorry, I couldn't fetch the weather data. Please check the location and try again."
    
    elif intent == "news_query":
        topic = entities.get("topic")
        if not topic:
            return "Sorry, I couldn't identify the topic. Please provide a valid topic."
        news_data = await get_news(topic)
        if news_data.get("articles"):
            headlines = [article["title"] for article in news_data["articles"][:5]]
            return f"Top news headlines for {topic}:\n" + "\n".join(headlines)
        else:
            return "Sorry, I couldn't fetch the news data. Please check the topic and try again."
    
    else:
        return "Sorry, I can only help with weather and news queries."

# Example usage
async def main():
    user_prompt = "What's the weather in New York?"
    response = await process_user_input(user_prompt)
    print(response)

    user_prompt = "Tell me the latest news about technology."
    response = await process_user_input(user_prompt)
    print(response)

asyncio.run(main())

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def process_user_input(prompt: str) -> str:
    try:
        intent, entities = extract_intent_and_entities(prompt)
        logger.info(f"Extracted intent: {intent}, entities: {entities}")
        
        if intent == "weather_query":
            location = entities.get("location")
            if not location:
                return "Sorry, I couldn't identify the location. Please provide a valid location."
            weather_data = get_weather(location)
            if weather_data.get("weather"):
                description = weather_data["weather"][0]["description"]
                temp = weather_data["main"]["temp"]
                return f"The weather in {location} is currently {description} with a temperature of {temp}°K."
            else:
                return "Sorry, I couldn't fetch the weather data. Please check the location and try again."
        
        elif intent == "news_query":
            topic = entities.get("topic")
            if not topic:
                return "Sorry, I couldn't identify the topic. Please provide a valid topic."
            news_data = get_news(topic)
            if news_data.get("articles"):
                headlines = [article["title"] for article in news_data["articles"][:5]]
                return f"Top news headlines for {topic}:\n" + "\n".join(headlines)
            else:
                return "Sorry, I couldn't fetch the news data. Please check the topic and try again."
        
        else:
            return "Sorry, I can only help with weather and news queries."
    except Exception as e:
        logger.error(f"Error processing user input: {e}")
        return "An error occurred while processing your request. Please try again later."